In [1]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import numpy as np

In [ ]:
lfw_data = fetch_lfw_people(min_faces_per_person=70, resize=0.5)

In [ ]:
print("Shape of data:", lfw_data.data.shape)
print("Shape of target labels:", lfw_data.target.shape)

Shape of data: (1288, 2914)
Shape of target labels: (1288,)


In [ ]:
X = lfw_data.images  # shape: (n_samples, height, width)
y = lfw_data.target
target_names = lfw_data.target_names

In [ ]:
X[0]

array([[0.9973857 , 0.9973857 , 0.99607843, ..., 0.26928106, 0.23267974,
        0.20261438],
       [0.9973857 , 0.99607843, 0.99477124, ..., 0.275817  , 0.24052288,
        0.20915033],
       [0.9882353 , 0.97647065, 0.96732026, ..., 0.26928106, 0.24052288,
        0.21830066],
       ...,
       [0.3372549 , 0.2784314 , 0.20522876, ..., 0.4117647 , 0.39869282,
        0.37908497],
       [0.30980393, 0.2522876 , 0.19738562, ..., 0.39607847, 0.39607844,
        0.37254906],
       [0.28496733, 0.24705882, 0.19869281, ..., 0.38431373, 0.3869281 ,
        0.3803922 ]], dtype=float32)

In [ ]:
X

array([[[0.9973857 , 0.9973857 , 0.99607843, ..., 0.26928106,
         0.23267974, 0.20261438],
        [0.9973857 , 0.99607843, 0.99477124, ..., 0.275817  ,
         0.24052288, 0.20915033],
        [0.9882353 , 0.97647065, 0.96732026, ..., 0.26928106,
         0.24052288, 0.21830066],
        ...,
        [0.3372549 , 0.2784314 , 0.20522876, ..., 0.4117647 ,
         0.39869282, 0.37908497],
        [0.30980393, 0.2522876 , 0.19738562, ..., 0.39607847,
         0.39607844, 0.37254906],
        [0.28496733, 0.24705882, 0.19869281, ..., 0.38431373,
         0.3869281 , 0.3803922 ]],

       [[0.14509805, 0.1633987 , 0.21437909, ..., 0.22352941,
         0.1751634 , 0.13333334],
        [0.16601308, 0.21830066, 0.24183007, ..., 0.22875817,
         0.21045752, 0.16470589],
        [0.18169935, 0.25490198, 0.2901961 , ..., 0.20130719,
         0.20784314, 0.16209151],
        ...,
        [0.2888889 , 0.29673204, 0.3006536 , ..., 0.48104575,
         0.4431373 , 0.45751634],
        [0.2

In [ ]:
target_names

array(['Ariel Sharon', 'Colin Powell', 'Donald Rumsfeld', 'George W Bush',
       'Gerhard Schroeder', 'Hugo Chavez', 'Tony Blair'], dtype='<U17')

In [ ]:
# Normalize pixel values
X = X / 255.0


In [ ]:
X = np.expand_dims(X, axis=-1)  # shape becomes (n_samples, height, width, 1)


In [ ]:
# One-hot encode labels
lb = LabelBinarizer()
y_encoded = lb.fit_transform(y)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
IMG_SIZE = X.shape[1]  # height and width are equal
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 1)
OUTPUT_SHAPE = len(target_names)


In [ ]:
INPUT_SHAPE

(62, 62, 1)

### let's build the model

In [ ]:
OUTPUT_SHAPE

7

In [ ]:
def create_face_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE):
    print("Building face classification model...")

    # Base model: simple CNN instead of MobileNetV2
    base_model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.GlobalAveragePooling2D()  # Similar to pooling="avg" in MobileNetV2
    ])

    # Freeze base model (optional here since it's custom)
    base_model.trainable = False

    # Final model
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dense(units=output_shape, activation="softmax")
    ])

    # Compile
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"]
    )

    return model

In [ ]:
model = create_face_model()
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Building face classification model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - accuracy: 0.3840 - loss: 1.9418 - val_accuracy: 0.4612 - val_loss: 1.9258
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4147 - loss: 1.9246 - val_accuracy: 0.4612 - val_loss: 1.9066
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3835 - loss: 1.9116 - val_accuracy: 0.4612 - val_loss: 1.8895
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3879 - loss: 1.8968 - val_accuracy: 0.4612 - val_loss: 1.8720
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4029 - loss: 1.8824 - val_accuracy: 0.4612 - val_loss: 1.8557
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3983 - loss: 1.8667 - val_accuracy: 0.4612 - val_loss: 1.8403
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4086 - loss: 1.8540 - val_accuracy: 0.4612 - val_loss: 1.8257
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3901 - loss: 1.8506 - val_accuracy: 0.4612 - val_loss

In [ ]:
target_names[np.argmax(y_test[200])]

np.str_('George W Bush')

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

# Predict on a sample
sample = X_test[200:201]
pred = model.predict(sample)
print("Predicted label:", target_names[np.argmax(pred)])

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5007 - loss: 1.7673 
Test Accuracy: 0.46
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 681ms/step
Predicted label: George W Bush


In [ ]:
# Define your target path
save_path = "/content/drive/MyDrive/attendace_By_face/lfw_face_classifier.h5"

# Save the model
model.save(save_path)
print(f"Model saved to: {save_path}")

Model saved to: /content/drive/MyDrive/attendace_By_face/lfw_face_classifier.h5


### function to convert the raw image data into tensors

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

def preprocess_input_image(img: Image.Image, target_size=(62, 47)) -> np.ndarray:
    """
    Converts a PIL image to the same format as LFW dataset:
    - Resize to (62, 47) as per LFW default
    - Convert to grayscale
    - Normalize pixel values to [0, 1]
    - Expand dimensions to match model input shape
    """
    # Resize and convert to grayscale
    img = img.resize(target_size).convert("L")  # "L" mode = grayscale

    # Convert to NumPy array and normalize
    img_array = np.array(img) / 255.0

    # Expand dimensions: (height, width) → (height, width, 1)
    img_array = np.expand_dims(img_array, axis=-1)

    # Add batch dimension: (height, width, 1) → (1, height, width, 1)
    img_array = np.expand_dims(img_array, axis=0)

    return img_array.astype(np.float32)

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/attendace_By_face/lfw_face_classifier.h5")

In [ ]:
# Execute the cell to save weights
model.save_weights("/content/drive/MyDrive/attendace_By_face/lfw_face_classifier.weights.h5")

In [ ]:
#### creating the attendance file
# import csv
# import os

# ATTENDANCE_FILE = "/content/drive/MyDrive/attendace_By_face/attendance.csv"

# def initialize_attendance_sheet(target_names):
#     with open(ATTENDANCE_FILE, mode="w", newline="") as file:
#         writer = csv.writer(file)
#         writer.writerow(["Name", "Status"])
#         for name in target_names:
#             writer.writerow([name, "Absent"])
#     print("Attendance sheet initialized with all names marked as Absent.")

In [ ]:
# Load trained model  if saved
model = create_face_model()
model.load_weights("/content/drive/MyDrive/attendace_By_face/lfw_face_classifier.weights.h5")


# Predict from uploaded image
def predict_person_name(image):
    processed = preprocess_input_image(image)
    prediction = model.predict(processed)
    predicted_index = np.argmax(prediction)
    return target_names[predicted_index]

def predict_person_name(image):
    processed = preprocess_input_image(image)
    prediction = model.predict(processed)
    predicted_index = np.argmax(prediction)
    name = target_names[predicted_index]

    # Mark attendance with duplicate check
    mark_attendance(name)

    return f"{name} processed for attendance"

# def predict_person_name(image):
#     processed = preprocess_input_image(image)
#     prediction = model.predict(processed)
#     predicted_index = np.argmax(prediction)
#     name = target_names[predicted_index]

#     # Mark as present in attendance sheet
#     mark_present(name)

#     return f"{name} marked as Present"

Building face classification model...


In [ ]:
import csv
from datetime import datetime
import os

ATTENDANCE_FILE = "/content/drive/MyDrive/attendace_By_face/attendance.csv"

def mark_attendance(name, status="Present"):
    today = datetime.now().strftime("%Y-%m-%d")
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Create file with header if it doesn't exist
    if not os.path.exists(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Timestamp", "Status"])

    # Check for existing entry today
    already_marked = False
    with open(ATTENDANCE_FILE, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["Name"] == name and row["Timestamp"].startswith(today):
                already_marked = True
                break

    # Append if not already marked
    if not already_marked:
        with open(ATTENDANCE_FILE, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([name, timestamp, status])
        print(f"{name} marked as Present at {timestamp}")
    else:
        print(f"{name} is already marked Present for today.")


# def mark_present(name):
#     rows = []
#     with open(ATTENDANCE_FILE, mode="r") as file:
#         reader = csv.reader(file)
#         header = next(reader)
#         for row in reader:
#             if row[0] == name:
#                 row[1] = "Present"
#             rows.append(row)

#     with open(ATTENDANCE_FILE, mode="w", newline="") as file:
#         writer = csv.writer(file)
#         writer.writerow(header)
#         writer.writerows(rows)
#     print(f"{name} marked as Present.")

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=predict_person_name,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="LFW Human Face Classifier",
    description="Upload a face image to identify the person using a model trained on the LFW dataset."
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4bcb3f1190559c170d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install -q huggingface_hub
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from huggingface_hub import upload_file, HfFolder

upload_file(
    path_or_fileobj="/content/drive/MyDrive/attendace_By_face/lfw_face_classifier.h5",            # local Colab model path
    path_in_repo="lfw_face_classifier.h5",               # name it will have in the repo
    repo_id="kirito0o8/attendance-by-face",    # your repo ID
    token=HfFolder.get_token()
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...dace_By_face/lfw_face_classifier.h5: 100%|##########|  411kB /  411kB            

CommitInfo(commit_url='https://huggingface.co/kirito0o8/Attendance-by-face/commit/110d19dc3868d9505a73c5f6a3ce591cd6ff7235', commit_message='Upload lfw_face_classifier.h5 with huggingface_hub', commit_description='', oid='110d19dc3868d9505a73c5f6a3ce591cd6ff7235', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kirito0o8/Attendance-by-face', endpoint='https://huggingface.co', repo_type='model', repo_id='kirito0o8/Attendance-by-face'), pr_revision=None, pr_num=None)

In [ ]:
!hf download kirito0o8/Attendance-by-face --repo-type=space

Fetching 2 files: 100% 2/2 [00:00<00:00, 11915.64it/s]
/root/.cache/huggingface/hub/spaces--kirito0o8--Attendance-by-face/snapshots/194c9177eb1536aaa5092a90961f6f8f797f1bb7


In [ ]:
!git clone https://huggingface.co/spaces/kirito0o8/attendance-by-face

Cloning into 'attendance-by-face'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (8/8), 2.68 KiB | 2.68 MiB/s, done.


In [ ]:
%cd attendance-by-face

/content/attendance-by-face/attendance-by-face/attendance-by-face/attendance-by-face/attendance-by-face/attendance-by-face/attendance-by-face


In [ ]:
%%writefile requirements.txt
gradio
tensorflow
numpy
Pillow

Writing requirements.txt


In [ ]:
from huggingface_hub import hf_hub_download
import shutil

# Download from your model repo
model_path = hf_hub_download(repo_id="kirito0o8/attendance-by-face", filename="lfw_face_classifier.h5")

# Copy into your Space repo
shutil.copy(model_path, "lfw_face_classifier.h5")

'lfw_face_classifier.h5'

In [ ]:
!pip install tensorflow gradio

In [ ]:
%%writefile app.py
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import csv
import os

# Load model
model_path = "lfw_face_classifier.h5"
model = tf.keras.models.load_model(model_path)

# Define known class names (replace with actual names if needed)
target_names = ["George W Bush", "Colin Powell", "Donald Rumsfeld", "Tony Blair", "Condoleezza Rice"]

# Attendance file path
ATTENDANCE_FILE = "attendance.csv"

# Preprocess uploaded image
def preprocess_input_image(img: Image.Image, target_size=(62, 47)) -> np.ndarray:
    img = img.resize(target_size).convert("L")
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=-1)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array.astype(np.float32)

# Initialize attendance sheet
def initialize_attendance_sheet():
    if not os.path.exists(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Status"])
            for name in target_names:
                writer.writerow([name, "Absent"])

# Mark person as present
def mark_present(name):
    rows = []
    with open(ATTENDANCE_FILE, mode="r") as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            if row[0] == name:
                row[1] = "Present"
            rows.append(row)
    with open(ATTENDANCE_FILE, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(rows)

# Prediction + attendance
def predict(image):
    initialize_attendance_sheet()
    processed = preprocess_input_image(image)
    prediction = model.predict(processed)
    predicted_index = np.argmax(prediction)
    name = target_names[predicted_index]
    mark_present(name)
    return f"{name} marked as Present"

# Gradio interface
gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Face Attendance Classifier",
    description="Upload a face image to identify the person and mark attendance using a model trained on the LFW dataset."
).launch()

Overwriting app.py


In [ ]:
!git config user.email "ktapaskumar666@gmail.com"
!git config user.name "tapas-kumar-shety"


In [ ]:
!git add .
!git commit -m "Add app.py, class names, and requirements"
!git push

[main 8b11f77] Add app.py, class names, and requirements
 1 file changed, 4 insertions(+)
 create mode 100644 requirements.txt
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 335 bytes | 335.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/spaces/kirito0o8/attendance-by-face
   b591a11..8b11f77  main -> main
